In [2]:
import pandas as pd
import pyspark
from operator import add
import os
from pyspark import SparkContext, SparkConf

In [3]:
conf = SparkConf().setAppName("Spark Apriori").setMaster(os.getenv('SPARK_URL'))
sc = SparkContext(conf=conf)

23/04/27 16:51:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
sc

<SparkContext master=spark://cm041:46605 appName=Spark Apriori>

In [19]:
# Read file
file =  sc.textFile("./dataset/apriori_ratings.txt")
file = file.map(lambda line: line.strip('"'))
print(file.collect()[:10])

['858 1246 1968 4226', '605 628', '1645 1909 1911 2023 2355', '858 1945', '293 318', '318 1266 2019', '1246 4226 2355 497 1262 2324', '500 318 1073 1923 4995 51540', '858 1246 2355 293 318 497 590 1089 176 322 441 492 501 549 608 926 953 1213 1273 1537 1635 1639 1683 1883 1957 2020 2064 2095 2348 2657', '293 8961 1265 1262 608 745 4327 6947 7445 8874']


In [20]:
file.getNumPartitions()

2

In [21]:
file.count()

163556

In [23]:
## Each transaction to be a list: [a, b, c] 
transactions = file.map(lambda line: line.split(' '))

print(transactions.collect()[:10])

[['858', '1246', '1968', '4226'], ['605', '628'], ['1645', '1909', '1911', '2023', '2355'], ['858', '1945'], ['293', '318'], ['318', '1266', '2019'], ['1246', '4226', '2355', '497', '1262', '2324'], ['500', '318', '1073', '1923', '4995', '51540'], ['858', '1246', '2355', '293', '318', '497', '590', '1089', '176', '322', '441', '492', '501', '549', '608', '926', '953', '1213', '1273', '1537', '1635', '1639', '1683', '1883', '1957', '2020', '2064', '2095', '2348', '2657'], ['293', '8961', '1265', '1262', '608', '745', '4327', '6947', '7445', '8874']]


In [24]:
## Flat all transactions into one list
flatTransactions = transactions.flatMap(lambda line: line)

# flatTransactions.collect()

In [25]:
## Get a list of all unique items 
uniqueItems = file.flatMap(lambda line:line.split(',')).distinct()

# print(uniqueItems.collect())

In [26]:
## Construct unique items as tuples -> (item, count)
supportRdd = flatTransactions.map(lambda item: (item , 1))
supportRdd = supportRdd.reduceByKey(add)
# supportRdd.collect()

In [27]:
## 1-item set support values
supports = supportRdd.map(lambda item: item[1])
# supports.collect()

In [28]:
# ## Unique frequent items in dataset
# uniqueItems = wlitems.distinct()

# # Add 1 as Tuple
# supportRdd = wlitems.map(lambda item: (item , 1))

# # Method for sum in reduceByKey method
# def sumOparator(x,y):
#     return x+y

# # Sum of values by key
# supportRdd = supportRdd.reduceByKey(sumOparator)

# print(supportRdd.collect()) # Retruns following array 
# [('Apple', 12), ('Mango', 10), ('Banana', 9), ('Coconut', 3), 
#  ('Strawberry', 4), ('Grapes', 2), ('Lemon', 1), ('Raspberry', 9), ('Rassberry', 1)]


# # First support values
# supports = supportRdd.map(lambda item: item[1]) # Return only support values

In [29]:
## Minimum support value
# minSupport = max(4, supports.min())
minSupport = 100

In [30]:
## Filter first supportRdd with minimum support: (item, support)
supportRdd = supportRdd.filter(lambda item: item[1] >= minSupport )
# supportRdd.collect()

## Create base RDD which will be updated every iteration
## put item[0] in a list: ([a, b, c], support)
baseRdd = supportRdd.map(lambda item: ([item[0]] , item[1])) 
# baseRdd.collect()

## supportRdd: frequent k-item set
supportRdd = supportRdd.map(lambda item: item[0]) # only store item set

In [31]:
baseRdd.count()

534

In [32]:
# # Define minimum support value 
# minSupport = supports.min()

# # If mininmum support is 1 then replace it with 2 
# minSupport = 2 if minSupport == 1 else minSupport

# ## Filter first supportRdd with minimum support 
# supportRdd = supportRdd.filter(lambda item: item[1] >= minSupport )

# ## Create base RDD which will be updated every iteration
# baseRdd = supportRdd.map(lambda item: ([item[0]] , item[1])) # put item[0] in a list
# print('1 . Table has crated...') 

# supportRdd = supportRdd.map(lambda item: item[0]) # only store itemset
# supportRddCart = supportRdd

In [33]:
def removeReplica(record):

    if(isinstance(record[0], tuple)):
        x1 = record[0]
        x2 = record[1]
    else:
        x1 = [record[0]]
        x2 = record[1]

    if(any(x == x2 for x in x1) == False):
        a = list(x1)
        a.append(x2)
        a.sort()
        result = tuple(a)
        return result 
    else:
        return x1

In [ ]:
c = 2 # Combination length 

while(supportRdd.isEmpty() == False):
    
    ## cartesian product of frequent k-item set and unique items -> candidate (k+1)-item set 
    combined = supportRdd.cartesian(uniqueItems) 
    combined = combined.map(lambda item: removeReplica(item))
  
    combined = combined.filter(lambda item: len(item) == c)
    combined = combined.distinct()

    
    combined_2 = combined.cartesian(transactions)
    combined_2 = combined_2.filter(lambda item: all(x in item[1] for x in item[0]))
    
#     combined_2 = combined_2.map(lambda item: item[0])
    combined_2 = combined_2.map(lambda item: (item[0], 1))
    combined_2 = combined_2.reduceByKey(add)
    combined_2 = combined_2.filter(lambda item: item[1] >= minSupport)

    baseRdd = baseRdd.union(combined_2)
    
    combined_2 = combined_2.map(lambda item: item[0])
    supportRdd = combined_2
    print(c ,'. Table has crated... ')
#     print(combined_2.collect())
    c = c+1 

2 . Table has crated... 


In [26]:
baseRdd.getNumPartitions()

170

In [27]:
baseRdd.count()

12

In [28]:
baseRdd.collect()

[(['Apple'], 12),
 (['Strawberry'], 4),
 (['Mango'], 10),
 (['Banana'], 9),
 (['Raspberry'], 10),
 (('Mango', 'Raspberry'), 4),
 (('Banana', 'Raspberry'), 4),
 (('Apple', 'Mango'), 7),
 (('Apple', 'Banana'), 5),
 (('Banana', 'Mango'), 6),
 (('Apple', 'Raspberry'), 4),
 (('Apple', 'Banana', 'Mango'), 4)]

In [21]:
def filterForConf(item , total):
    '''
    item: [(((a,b,c), 3), ((a,c), 2)), ...]
    this function filter `item` based on two conditions:
    1. the left has more items than the right
    2. any item in the right has a match in the left
    '''
    if(len(item[0][0]) > len(item[1][0])  ):
        if(checkItemSets(item[0][0] , item[1][0]) == False):
            pass
        else:
            return (item)       
    else:
        pass  

    
def checkItemSets(item_1 , item_2):

    if(len(item_1) > len(item_2)):
#             return all(right in item_1 for right in item_2)
        return all(any(k == l for k in item_1 ) for l in item_2)
    else:
        return all(left in item_1 for left in item_1)
#             return all(any(k == l for k in item_2 ) for l in item_1)


def calculateConfidence(item):

    # Parent item list
    parent = set(item[0][0])

    # Child item list
    if(isinstance(item[1][0] , str)):
        child  = set([item[1][0]])
    else:
        child  = set(item[1][0])
    # Parent and Child support values
    parentSupport = item[0][1]
    childSupport = item[1][1]
    # Finds the item set confidence is going to be found

    support = (parentSupport / childSupport)*100

    return list([ list(child) ,  list(parent.difference(child)) , support ])


# Example ((('x10', 'x3', 'x6', 'x7', 'x9'), 1), (('x10', 'x3', 'x7'), 1))
calcuItems = baseRdd.cartesian(baseRdd)

#deneme = calcuItems.map(lambda item: lens(item)) 
total = calcuItems.count()

print('# : Aggregated support values preparing for the confidence calculatations')
baseRddConfidence = calcuItems.filter(lambda item: filterForConf(item , total))
print('# : Aggregated support values are ready !')
baseRddConfidence = baseRddConfidence.map(lambda item: calculateConfidence(item))

  
# print(baseRddConfidence.collect())

## Create an array with collected baseRddConfidence results
result = baseRddConfidence.collect()

## Create Data Frame
confidenceTable = pd.DataFrame(data = result , columns=["Before", "After" , "Confidence"])

## Show data frame
print(confidenceTable)

# : Aggregated support values preparing for the confidence calculatations
# : Aggregated support values are ready !


                 Before                After  Confidence
0               [Apple]              [Mango]   58.333333
1               [Mango]              [Apple]   70.000000
2               [Apple]             [Banana]   41.666667
3               [Apple]            [Coconut]   16.666667
4              [Banana]              [Apple]   55.555556
5             [Coconut]              [Apple]   66.666667
6               [Apple]         [Strawberry]   25.000000
7               [Apple]          [Raspberry]   33.333333
8               [Mango]             [Banana]   60.000000
9              [Banana]              [Mango]   66.666667
10         [Strawberry]              [Apple]   75.000000
11          [Raspberry]              [Apple]   40.000000
12             [Grapes]          [Raspberry]  100.000000
13          [Raspberry]             [Grapes]   20.000000
14              [Mango]          [Raspberry]   40.000000
15             [Banana]          [Raspberry]   44.444444
16          [Raspberry]        

In [17]:
class Filter():

    def __init__(self):
        
        self.stages = 1


    def filterForConf(self, item , total):
        
        if(len(item[0][0]) > len(item[1][0])  ):
            if(self.checkItemSets(item[0][0] , item[1][0]) == False):
                pass
            else:
                return (item)       
        else:
            pass  
        self.stages = self.stages + 1

    # Check Items sets includes at least one comman item // Example command: # any(l == k for k in z for l in x )
    def checkItemSets(self, item_1 , item_2):

        if(len(item_1) > len(item_2)):
            return all(any(k == l for k in item_1 ) for l in item_2)
        else:
            return all(any(k == l for k in item_2 ) for l in item_1)


    def calculateConfidence(self, item):

        # Parent item list
        parent = set(item[0][0])
        
        # Child item list
        if(isinstance(item[1][0] , str)):
            child  = set([item[1][0]])
        else:
            child  = set(item[1][0])
        # Parent and Child support values
        parentSupport = item[0][1]
        childSupport = item[1][1]
        # Finds the item set confidence is going to be found

        support = (parentSupport / childSupport)*100

        return list([ list(child) ,  list(parent.difference(child)) , support ])

        
# Example ((('x10', 'x3', 'x6', 'x7', 'x9'), 1), (('x10', 'x3', 'x7'), 1))
calcuItems = baseRdd.cartesian(baseRdd)

# Create Filter Object
ff = Filter()

#deneme = calcuItems.map(lambda item: lens(item)) 
total = calcuItems.count()

print('# : Aggregated support values preparing for the confidence calculatations')
baseRddConfidence = calcuItems.filter(lambda item: ff.filterForConf(item , total))
print('# : Aggregated support values are ready !')
baseRddConfidence = baseRddConfidence.map(lambda item: ff.calculateConfidence(item))

  
# print(baseRddConfidence.collect())

## Create an array with collected baseRddConfidence results
result = baseRddConfidence.collect()

## Create Data Frame
confidenceTable = pd.DataFrame(data = result , columns=["Before", "After" , "Confidence"])

## Show data frame
print(confidenceTable)

# : Aggregated support values preparing for the confidence calculatations
# : Aggregated support values are ready !


                 Before                After  Confidence
0               [Apple]              [Mango]   58.333333
1               [Mango]              [Apple]   70.000000
2               [Apple]             [Banana]   41.666667
3               [Apple]            [Coconut]   16.666667
4              [Banana]              [Apple]   55.555556
5             [Coconut]              [Apple]   66.666667
6               [Apple]         [Strawberry]   25.000000
7               [Apple]          [Raspberry]   33.333333
8               [Mango]             [Banana]   60.000000
9              [Banana]              [Mango]   66.666667
10         [Strawberry]              [Apple]   75.000000
11          [Raspberry]              [Apple]   40.000000
12             [Grapes]          [Raspberry]  100.000000
13          [Raspberry]             [Grapes]   20.000000
14              [Mango]          [Raspberry]   40.000000
15             [Banana]          [Raspberry]   44.444444
16          [Raspberry]        